#### Lab 7 
#### Rouge Metrics

In [0]:
pip install python-levenshtein


     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144800 sha256=962e7c6fcef02abdbbe822e2d52a104b5a947ca952639f919eba07f68c7aa820
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [0]:
pip install fuzzywuzzy


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "https://raw.githubusercontent.com/shynybayev/NLP-Labs/master/Lab7/data.csv"

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df = pd.read_csv(__PATH__,sep=";",header=0)

#### Preprocessing the title to list of tokens

In [0]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [0]:
res = {}
for title in titles:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets

In [0]:
def extractHprnmsEng(a):
    hprnms1_eng = set()
    hprnms2_eng = set()
    counter = 1

    for wrd in a:
        try:
            snst = wn.synsets(wrd)[0]
            while counter <= 2:
                snst = snst.hypernyms()[0]
            if(counter == 1):
              hprnms1_eng.add(snst)
            elif(counter == 2):
              hprnms2_eng.add(snst)
            counter+=1
        except IndexError:
            continue
            print('some exception')
    return hprnms1_eng, hprnms2_eng

def fMeasure(a, b):
    a = set(a)
    b = set(b)
    overlap = set()
    count = 0
    overlap = a.intersection(b)
    count = len(overlap) 
    recall = count/len(a)
    precision = count/len(b)
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recall*precision/(recall+precision), overlap
    


In [0]:
lang = 'eng'

def distance(a, b):
    a = set(a) 
    b = set(b)
    count = 0
    f1score, overlap = fMeasure(a, b)
    a = a - overlap
    b = b - overlap
    if len(a) == 0 or len(b) == 0:
        return 1.0 - f1score
    buff1_a = {}
    buff2_a = {}
    buff1_b = {}
    buff2_b = {}
    if lang == 'eng':
        buff1_a, buff2_a = extractHprnmsEng(a)
        buff1_b, buff2_b = extractHprnmsEng(b)
    if len(buff1_a.intersection(buff1_b)) > 0:
        count += 1
    if len(buff2_a.intersection(buff2_b)) > 0:
        count += 0.5
    recall = count/len(a)
    precision = count/len(b)
    f1score_hyp = 2*recall/(recall + precision) if count > 0 else 0
    f1res = (2*f1score+f1score_hyp)/3
    return (1.0 - f1res)

buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri]=distance(ll[0],rr[0])
dist
        

array([[0.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        0.91111111],
       [1.        , 1.        , 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.91111111, 1.        ,
        0.        ]])

#### Top ten closest articles with fuzzy metrics of titles

---



In [0]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [0]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…